In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\super\AppData\Roa

In [2]:
#loading the model and storing in model variable
model=load_model(r'C:\\Users\\super\\handEmoji.h5')

W1013 21:57:40.088296  9124 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1013 21:57:40.121555  9124 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1013 21:57:40.149755  9124 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1013 21:57:40.240723  9124 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1013 21:57:40.243853  9124 deprecation_wrapper.py:119] From C:\ProgramData\Ana

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji=cv2.resize(emoji,(w,h))  
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image


def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))
#keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    if ret:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img,emojis[pred_class], 400, 250, 90, 90)

    x, y, w, h = 300, 50, 350, 350
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    if cv2.waitKey(27) & 0xFF==ord('q'):
        break 


0
1
2
3
4
5
6
7
8
9
10
11
12
1 1.0
10 0.99999976
11 1.0
3 1.0
3 1.0
3 1.0
10 1.0
11 1.0
1 1.0
4 1.0
6 1.0
10 1.0
6 1.0
10 1.0
9 1.0
10 1.0
3 1.0
6 1.0
4 1.0
1 1.0
5 0.9995534
9 1.0
6 1.0
6 1.0
11 1.0
6 1.0
6 1.0
3 1.0
6 1.0
7 1.0
10 1.0
10 1.0
6 1.0
4 1.0
10 1.0
3 1.0
10 1.0
6 1.0
6 1.0
6 1.0
10 1.0
1 1.0
6 1.0
6 1.0
3 1.0
10 1.0
6 1.0
10 1.0
10 1.0
6 1.0
9 1.0
4 1.0
10 0.98298025
10 1.0
6 1.0
10 1.0
1 1.0
3 1.0
4 1.0
6 1.0
6 1.0
6 1.0
6 1.0
6 1.0
6 1.0
9 1.0
3 1.0
3 1.0
1 1.0
6 1.0
10 1.0
6 1.0
6 1.0
1 1.0
3 1.0
9 1.0
10 1.0
10 1.0
10 1.0
10 1.0
3 1.0
3 0.9999882
11 1.0
6 1.0
10 1.0
6 0.9998702
6 1.0
9 1.0
6 1.0
6 1.0
3 1.0
6 1.0
6 1.0
6 0.7623135
5 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
8 1.0
11 1.0
11 1.0
8 0.9305767
11 1.0
11 1.0
8 0.9999995
8 1.0
8 1.0
11 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
11 1.0
11 1.0
5 1.0
2 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 

1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 0.99600923
1 0.99999917
8 1.0
1 1.0
1 1.0
1 1.0
8 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
8 0.9999927
1 1.0
1 1.0
1 1.0
8 1.0
8 1.0
1 0.9744864
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 0.99999976
8 1.0
8 1.0
3 1.0
8 0.99999285
8 1.0
8 0.9999999
8 1.0
8 1.0
1 1.0
1 1.0
4 1.0
4 1.0
6 1.0
6 1.0
6 1.0
6 1.0
5 1.0
5 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
5 0.9999993
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
5 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
5 1.0
5 0.7216942
5 1.0
7 1.0
7 1.0
7 1.0


6 1.0
6 1.0
6 1.0
6 1.0
1 1.0
1 1.0
1 1.0
1 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
11 1.0
7 1.0
7 1.0
7 1.0
7 0.9999999
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
11 1.0
11 1.0
7 1.0
7 1.0
11 0.9996841
11 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
4 1.0
4 1.0
4 1.0
7 0.9975508
7 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
7 1.0
7 1.0
7 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
11 1.0
11 1.0
11 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
11 1.0
11 1.0
11 1.0
7 1.0
7 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
3 1.0
3 1.0
3 1.0
5 1.0
3 1.0
6 1.0
7 1.0
7

8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
1 0.6531269
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
1 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
1 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
1 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
8 1.

3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
2 1.0
3 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.

9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
1 1.0
9 1.0
9 1.0
1 1.0
1 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
1 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
1 0.96533626
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1

1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
11 1.0
6 0.9929242
2 1.0
2 1.0
2 1.0
3 0.9999999
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
2 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
4 1.0
3 1.0
4 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
5 1.0
4 0.9999999
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
4 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
7 1.0
3 1.0
3 1.0
3 1.0
3 1.0
3 1.0
7 1.0
7 1.0
9 1.0
1 1.0
3 1.0
3 1.0
3 1.0
1 0.99999917
3 1.0
3 1.0
8 1.0
8 1.0
1 1.0
1 1.0
6 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
8 1.0
1 1.0
8 1.0
8 1.0
1 0.99993277
8 1.0
8 1.0
8 1.0
8 1.0
8 1.0
1 1.0
1 1.0
1 1.0
1